In [14]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))

In [16]:
# Problem 1
TP = 12*60*60
mu = 3.986004354360959e5
J2 = 1.08262668e-3
Re = 6378
rp = 600 + Re
i = 63*np.pi/180
a = ((TP*np.sqrt(mu))/2*np.pi)**(2/3)
e = 1 - rp/a
p = a*(1-e**2)
n = 2*np.pi/TP
delta_omega = (3/4)*J2*n*(Re/p)**2*(5*cos(i)**2-1)
delta_v = -60*(e*np.sqrt(mu/p))*(3*np.pi*J2/2)*(Re/p)**2*(5*cos(i)**2-1)
displayVariableWithUnits('a',a,'km')
displayVariable('e',e)
displayVariableWithUnits('p',p,'km')
displayVariableWithUnits('\\Delta{\\omega}',delta_omega*30*24*60*60*180/np.pi,'degrees')
displayVariableWithUnits('\\Delta{V}',abs(delta_v),'\\frac{km}{s}')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [179]:
# Problem 2
Omega_sun_sync =  1.991063853e-7
rp = 600 + Re
alpha = lambda a,e: (-3/2)*J2*np.sqrt(mu/a**3)*(Re/(a*(1-e**2)))**2
f = lambda a,e,i: alpha(a,e)*np.cos(i) - Omega_sun_sync
f_prime = lambda a,e,i: -alpha(a,e)*np.sin(i)
g = lambda a,e: a*(1-e) - rp
g_partial_a = lambda e : 1-e
g_partial_e = lambda a: -a
tolerance = 1e-10
i_guesses = [np.pi/2-.001,3*np.pi/2+0.001]
newton_rapson1 = {'i':[],'a':[],'e':[],'f':[],'g':[]}
newton_rapson2 = {'i':[],'a':[],'e':[],'f':[],'g':[]}
# Sorry for the Nesting the newton raphson method
# Im checking every possible elliptical orbit and both values of inclination
# Then I noticed that all the values seem to converge no matter what e is
# So Im taking the average of both instances of i,a
for ek in np.linspace(0.01,.99,1000):
  for guess_attempt,ik in enumerate(i_guesses):
    ak = rp
    while abs(f(ak,ek,ik))>tolerance or abs(g(ak,ek))>tolerance:
      ak = ak - g(ak,ek)/(g_partial_a(ek)+g_partial_e(ak))
      ek = ek - g(ak,ek)/(g_partial_e(ak)+g_partial_a(ek))
      ik = ik - f(ak,ek,ik)/f_prime(ak,ek,ik)
      if abs(ik) > 2*np.pi:
        print('error: divergent series')
        break
    if guess_attempt == 0 and not abs(ik) > 2*np.pi :
      newton_rapson1['i'].append(ik),newton_rapson1['a'].append(ak),newton_rapson1['e'].append(ek),newton_rapson1['f'].append(f(ak,ek,ik)),newton_rapson1['g'].append(g(ak,ek))
    if guess_attempt == 1 and not abs(ik) > 2*np.pi:
      newton_rapson2['i'].append(ik),newton_rapson2['a'].append(ak),newton_rapson2['e'].append(ek),newton_rapson2['f'].append(f(ak,ek,ik)),newton_rapson2['g'].append(g(ak,ek))
pd.DataFrame().from_dict(newton_rapson1).to_csv('Finding1stInclination.csv')
pd.DataFrame().from_dict(newton_rapson2).to_csv('Finding2ndInclination.csv')
i_1_degrees = (sum(newton_rapson1['i']) / len(newton_rapson1['i']))*180/np.pi
i_2_degrees = (sum(newton_rapson2['i']) / len(newton_rapson2['i']))*180/np.pi
a_1 = (sum(newton_rapson1['a']) / len(newton_rapson1['a']))
a_2 = (sum(newton_rapson2['a']) / len(newton_rapson2['a']))
a = (a_1+a_2)/2
displayVariableWithUnits('i_1',i_1_degrees,'degrees')
displayVariableWithUnits('i_2',i_2_degrees,'degrees')
displayVariableWithUnits('a',a,'km')
displayVariable('e',abs(1-rp/a))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>